In [1]:
import cv2
import numpy as np
import time 
import matplotlib.pyplot as plt

In [2]:
 # 根据路径读取一张图片
start=time.clock()    
image = cv2.imread("left0036.jpg")
shapes=image.shape
print('shapes',shapes)
# 剪切有效区域 ：判断图片的大小，是否需要裁剪
images_size=[shapes[0],shapes[1]]
images_size=np.min(images_size)
if images_size > 1000:
    image=image[0:int(1/3*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
elif images_size > 800:
    image=image[int(1/6*shapes[1]):int(5/6*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
else:
    image=image
cv2.imshow('image',image)
# 计算有效区域的面积
shapes=image.shape
propotion=image.shape[0]*image.shape[1]
 # BGR转HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#保留底片
end=image.copy()

shapes (600, 800, 3)


In [3]:
# 绿色的范围
def getGreen(image):    
    lower_green = np.array([45, 70, 100])
    upper_green = np.array([85, 255, 255])
    return {'green':getRange(image,lower_green,upper_green)}

# 红色的范围
def getRed(image):    
    low_red = np.array([0, 100, 80])
    high_red = np.array([8, 255, 255])
    return {'red':getRange(image,low_red,high_red)}

# 绿色的范围
def getYellow(image):    
    low_yellow = np.array([10, 100, 220])
    high_yellow = np.array([45, 255, 255])
    return {'yellow':getRange(image,low_yellow,high_yellow)}

def getRange(image,low,high):
# cv2.inRange函数设阈值，去除背景部分
    mask = cv2.inRange(image, low, high)
    return mask

def deNoise(img):   
    erode=cv2.erode(img,(100,100),iterations=4)
#降噪（模糊处理用来减少瑕疵点）
    blur = cv2.blur(erode, (5,5))   
# 膨胀
    dilate=cv2.dilate(blur,(100,100),iterations=4)
# 边缘检测    
    canny = cv2.Canny(blur, 150, 240)     
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
# 闭操作    
    closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
#     cv2.imshow('closed',closed)
    return closed

def getPositions(img):
    ret,thresh = cv2.threshold(img,127,255,0)
    tmp,contours,hierarchy=cv2.findContours(thresh,1,2)
#有效交通灯的比例    
    hsvList=[]
    if contours :
# 在单个颜色ROIs中hsv里面最亮的区域
        for i in range(len(contours)):
            rect=cv2.minAreaRect(contours[i])
            cnt=contours[i] 
            x,y,w,h=cv2.boundingRect(cnt)
            cv2.rectangle(end,(x,y),(x+w,y+h),(0,255,0),2)
            HSV=end[y:y+h,x:x+w,:]
            HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
            H, S, V = cv2.split(HSV)
            v = np.mean(V)
            hsvList.append((i,v,(y,y+h,x,x+w)))
        hsvList=sorted(hsvList,key=lambda x : (x[1]),reverse=True)     
        if len(hsvList)>0:            
            order=hsvList[0][0]
            coordinate=hsvList[0][2]
            return list(coordinate)
        else:
            return 0
    else:
        return 0
    


In [4]:
colors=[]
colors.append(getGreen(hsv))
colors.append(getRed(hsv))
colors.append(getYellow(hsv))
# 节省时间复杂度

areas=[]
sumResult=0
_max={'color':0}
# 三个区域最亮的地方
for i in range(len(colors)):
    closed = deNoise(list(colors[i].values())[0])
    coordinate = getPositions(closed)
    if isinstance(coordinate,list):
        print('coordinate',coordinate)
        HSV=end[coordinate[0]:coordinate[1],coordinate[2]:coordinate[3],:]
        HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
        H, S, V = cv2.split(HSV)
        v = np.mean(V)
        print('v',v)
        if v >list( _max.values())[0]:
            _max['color']=v
            _max[list(colors[i].keys())[0]] = _max.pop('color')
        sumResult+=v
        areas.append((list(colors[i].keys())[0],v))
    else:
        areas.append((list(colors[i].keys())[0],0))
        
# 按颜色值的大小排序
areas=sorted(areas,key=lambda x : (x[1]),reverse=True)
print(areas)
print(_max)
ended=time.clock()
print('total:',ended-start)
if int(areas[0][1]) != 0:
    print('The current color is:',areas[0][0])
else:
    print('没有交通灯')
cv2.waitKey(0)
cv2.destroyAllWindows()

coordinate [67, 82, 434, 448]
v 250.033333333
[('red', 250.03333333333333), ('green', 0), ('yellow', 0)]
{'red': 250.03333333333333}
total: 0.2808523770006156
The current color is: red
